# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as pd # pandas in Python
import Distributions as dst
import Plots as plts

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as pmf

### Odds

In [ ]:
function prob2odds(prob::Float64)::Float64
    @assert (0 <= prob <= 1)
    return prob / (1-prob)
end

In [ ]:
prob2odds(0.75)

In [ ]:
prob2odds(0.1)

In [ ]:
prob2odds(0.9)

In [ ]:
function odds2prob(odds::Float64)::Float64
   @assert 0 <= 0
   return odds / (odds + 1) 
end


For practice I will try to derive the formula (see odds2prob).
Let's go.

$odds = \frac{p}{1-p}$

$\frac{p}{1-p} = odds$

$p = odds * (1-p)$

$p = odds - odds*p$

$p + odds*p = odds$

$p * (1+odds) = odds$

$p = \frac{odds}{1+odds}$

$p = \frac{odds}{odds+1}$

In [ ]:
odds2prob(3/2)

In [ ]:
function odds2prob(yes::T, no::T)::Float64 where T<:Union{Int, Float64}
    @assert yes >= 0
    @assert no > 0
    return yes / (yes+no)
end

In [ ]:
odds2prob(3, 2)

In [ ]:
odds2prob(3/5, 2/5)

### Bayes's Rule

Bayes's theorem in the "probability form":

$P(H|D) = \frac{P(H) P(D|H)}{P(D)}$

Writing odds(A) for odds in favor of A, we can express Bayes's Theorem in "odds form":

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

The posterior odds are the prior odds times the likelihood ratio.

**My comment**

OK, although I'm not sure how we derived that and why we resigned from H and got A and B, all of a sudden.

Let's get back to the cookie problem. From Chapter 2 (The Bayes's Theorem).

We got two bowls:
- Bowl 1 contains 30 vanilla cookies and 10 chocolate cookies
- Bowl 2 contains 20 vanilla cookies and 20 chocolate cookies

You choose obe bowl at random and choose the cookie at random. If the cookie is vanilla, what is the probability that it came from Bowl 1?

The prior probability is 0.5, so the prior odds are 1 [odds(A)]

The likelihood ratio is 3/4 [P(D|A)] and 2/4 [P(D|B)]

The posterior odds are:

$odds(A|D) = 1 * \frac{3/4}{2/4}$

Which is equal to:


In [ ]:
1 * (3//4) / (2//4)

And now for the posterior probability

In [ ]:
odds2prob(3, 2)

If we draw another cookie and it's chocolate, we can do another update:

In [ ]:
3//2 * 1//4 / 1//2

In [ ]:
odds2prob(3, 4)